In [1]:
import pandas as pd
import numpy as np

import bbg_symbology as bbs
from bbg_loader_core import *

In [ ]:
def merge_dict(x, y):
    z = x.copy()   # start with x's keys and values
    z.update(y)    # modifies z with y's keys and values & returns None
    return z

In [4]:
INPUT_PATH = '../_in/'

FUTURES_MONTHS = {m:i+1 for i,m in enumerate('FGHJKMNQUVXZ')}


FutChainRef = FuturesChainReference(INPUT_PATH + 'futures_historical_chain.csv')
BbgTckrService = BloombergTckrService(FutChainRef)

'CLZ25 Comdty' in BTS.full_abb

In [3]:
FutChainService.get_futures_chain(alias='cl')

['CLM83 Comdty',
 'CLN83 Comdty',
 'CLQ83 Comdty',
 'CLU83 Comdty',
 'CLV83 Comdty',
 'CLX83 Comdty',
 'CLZ83 Comdty',
 'CLF84 Comdty',
 'CLG84 Comdty',
 'CLH84 Comdty',
 'CLJ84 Comdty',
 'CLK84 Comdty',
 'CLM84 Comdty',
 'CLN84 Comdty',
 'CLQ84 Comdty',
 'CLU84 Comdty',
 'CLV84 Comdty',
 'CLX84 Comdty',
 'CLZ84 Comdty',
 'CLF85 Comdty',
 'CLG85 Comdty',
 'CLH85 Comdty',
 'CLJ85 Comdty',
 'CLK85 Comdty',
 'CLM85 Comdty',
 'CLN85 Comdty',
 'CLQ85 Comdty',
 'CLU85 Comdty',
 'CLV85 Comdty',
 'CLX85 Comdty',
 'CLZ85 Comdty',
 'CLF86 Comdty',
 'CLG86 Comdty',
 'CLH86 Comdty',
 'CLJ86 Comdty',
 'CLK86 Comdty',
 'CLM86 Comdty',
 'CLN86 Comdty',
 'CLQ86 Comdty',
 'CLU86 Comdty',
 'CLV86 Comdty',
 'CLX86 Comdty',
 'CLZ86 Comdty',
 'CLF87 Comdty',
 'CLG87 Comdty',
 'CLH87 Comdty',
 'CLJ87 Comdty',
 'CLK87 Comdty',
 'CLM87 Comdty',
 'CLN87 Comdty',
 'CLQ87 Comdty',
 'CLU87 Comdty',
 'CLV87 Comdty',
 'CLX87 Comdty',
 'CLZ87 Comdty',
 'CLF88 Comdty',
 'CLG88 Comdty',
 'CLH88 Comdty',
 'CLJ88 Comdty

In [5]:
def bbg_to_alias_root(self, broot, yk):
    """convert bloomberg root, yellowkey to alias root"""
    sAlias = pd.Series([s for s in self._fut_ref.index])
    ix = pd.concat([self._fut_ref.Root == broot.lower(), self._fut_ref.YellowKey == yk], axis=1).all(axis=1)
    return self._fut_ref.loc[ix, 'Alias'][0]
